### Co-factor swap optimization

The p-coumaric acid production is NADPH dependent, meaning that a co-factor swap analysis might be useful to determine if the increasement of available NADPH by swapping it to NADH in other reactions could improve the production of our target molecule. This method relies on the fact, that oxidoreductases participating in the microbial metabolism posess
preferential binding specificity for either NADH or NADPH. Genreally, this also determines the role of a certain oxidoreductase enzyme: katabolic reactions are NAD+ or NADH dependent while anabolic ones rely on NADP+ or NADPH. OptSwap is a computational tool, which able to identify the optimal modification of cofactor binding specificities regarding to these enzymes, and previously were used succesfully in other settings (King et al., 2013).


In [43]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

In [44]:
model = read_sbml_model('iSynCJ816.xml')

#adding the metabolites (from KEGG, according to the article)
trans_Cinnamate = Metabolite(id = 'TRCI', compartment='c', name='trans-Cinnamate', formula='C9H8O2')
p_Coumaric_acid = Metabolite(id = 'PCOU', compartment='c', name='p-Coumaric acid', formula='C9H8O3')

#adding the reactions
phenylalanine_ammonia_lyase = Reaction('PAL')
trans_cinnamate_4_monooxygenase = Reaction('CYP73A')

#adding the metabolites into the reactions
phenylalanine_ammonia_lyase.add_metabolites(({model.metabolites.phe__L_c : -1,  
                              trans_Cinnamate: 1, 
                              model.metabolites.nh3_c: 1, 
                            }))
trans_cinnamate_4_monooxygenase.add_metabolites(({model.metabolites.o2_c : -1,
                                                  trans_Cinnamate: -1,
                                                  model.metabolites.nadph_c : -1,
                                                  p_Coumaric_acid: 1,
                                                  model.metabolites.h2o_c: 1,
                                                  model.metabolites.nadph_c : 1,
                                                 }))

#Adding the reactions to the model:
model.add_reactions([phenylalanine_ammonia_lyase])
model.add_reactions([trans_cinnamate_4_monooxygenase])

model.add_boundary(model.metabolites.TRCI, type='demand')
model.add_boundary(model.metabolites.PCOU, type='demand')

Reaction identifier,DM_PCOU
Name,p-Coumaric acid demand
Memory address,0x022b2df1edf0
Stoichiometry,PCOU --> p-Coumaric acid -->
GPR,
Lower bound,0
Upper bound,1000.0


In [45]:
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary.objective_functions import biomass_product_coupled_yield
from cameo.util import TimeMachine
from cameo.flux_analysis.analysis import flux_variability_analysis as fva

In [46]:
model.objective = model.reactions.CYP73A

In [47]:
model.reactions.BIOMASS_Ec_SynAuto_1.lower_bound = 0.1

In [48]:
py = product_yield(model.reactions.CYP73A, model.reactions.RBPC_1)

In [49]:
swap_optimization = CofactorSwapOptimization(model=model, objective_function=py)

In [50]:
result = swap_optimization.run(max_evaluations=2000, max_size=2)

Starting optimization at Mon, 28 Nov 2022 06:31:33


HBox()

Finished after 00:00:02


In [51]:
result

,index,targets,fitness


These results suggests, that unfortunately there is no target for cofactor specificity swapping to enhance the p-coumaric acid production.

### References:

[1] King, Zachary A., and Adam M. Feist. "Optimizing Cofactor Specificity of Oxidoreductase Enzymes for the Generation of Microbial Production Strains - OptSwap." Industrial Biotechnology 9, no. 4 (August 1,2013): 236-46. - doi:10.1089/ind. 2013.0005.